# [실험 재설계 2] '3' vs 'Not 3' 이진 분류 모델 분석

## 가설
1. **모델 A (가중치=1 초기화):** 은닉층의 대칭성 문제로 인해 '3'과 '3이 아닌 숫자'의 특징을 제대로 구분하지 못해 학습에 실패하고 정확도가 약 50% (찍기 수준) 근처에 머무를 것이다. (데이터 비율에 따라 다름)
2. **모델 B (표준 초기화):** '3'의 특징을 정상적으로 학습하여 높은 정확도로 이미지를 분류해낼 것이다.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.initializers import Initializer

try:
    import koreanize_matplotlib
except ImportError:
    !pip install koreanize_matplotlib
    import koreanize_matplotlib

### 1. 데이터 준비 (이진 분류를 위한 레이블 수정)

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 데이터 정규화
x_train, x_test = x_train / 255.0, x_test / 255.0

# '3'은 1 (True), 나머지는 0 (False)으로 이진 레이블 생성
y_train_binary = (y_train == 3).astype(int)
y_test_binary = (y_test == 3).astype(int)

print("훈련 데이터셋 레이블 예시:", y_train[:5])
print("이진 분류 훈련 레이블 예시:", y_train_binary[:5])
print(f"\n훈련 데이터 중 '3'의 비율: {np.mean(y_train_binary):.2%}")

훈련 데이터셋 레이블 예시: [5 0 4 1 9]
이진 분류 훈련 레이블 예시: [0 0 0 0 0]

훈련 데이터 중 '3'의 비율: 10.22%


In [ ]:
y_tra

### 2. 이진 분류 모델 생성
기존 784-128-64 구조는 유지하되, 출력층만 1개의 뉴런으로 변경합니다.

In [ ]:
# 모든 가중치를 1로 설정하는 커스텀 초기화 클래스
class OnesInitializer(Initializer):
    def __call__(self, shape, dtype=None):
        return tf.ones(shape, dtype=dtype)

def create_binary_classifier(initializer):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu', kernel_initializer=initializer, bias_initializer='zeros'),
        tf.keras.layers.Dense(64, activation='relu', kernel_initializer=initializer, bias_initializer='zeros'),
        # 출력층: '3일 확률' 하나만 출력하므로 뉴런 1개, 활성화 함수는 sigmoid
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# 모델 A: 은닉층 가중치를 1로 초기화
model_A = create_binary_classifier(OnesInitializer())

# 모델 B: 표준 무작위 초기화
model_B = create_binary_classifier('glorot_uniform')

# 두 모델을 동일한 조건으로 컴파일 (손실 함수로 binary_crossentropy 사용)
model_A.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_B.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_A.summary()

### 3. 모델 학습

In [ ]:
epochs = 15
batch_size = 128

print("모델 A 학습 시작...")
history_A = model_A.fit(x_train, y_train_binary,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(x_test, y_test_binary),
                        verbose=1)

print("\n모델 B 학습 시작...")
history_B = model_B.fit(x_train, y_train_binary,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(x_test, y_test_binary),
                        verbose=1)

### 4. 결과 시각화 및 분석

In [ ]:
plt.figure(figsize=(18, 6))

# 정확도 그래프
plt.subplot(1, 2, 1)
plt.plot(history_A.history['val_accuracy'], 'r-s', label='모델 A (가중치=1) 검증 정확도')
plt.plot(history_B.history['val_accuracy'], 'b-o', label='모델 B (표준 초기화) 검증 정확도')
plt.title('모델별 검증 정확도 비교')
plt.xlabel('에포크 (Epochs)')
plt.ylabel('정확도 (Accuracy)')
plt.ylim(0, 1.0) # Y축 범위를 0에서 1로 고정
plt.legend()
plt.grid(True)

# 손실 그래프
plt.subplot(1, 2, 2)
plt.plot(history_A.history['val_loss'], 'r-s', label='모델 A (가중치=1) 검증 손실')
plt.plot(history_B.history['val_loss'], 'b-o', label='모델 B (표준 초기화) 검증 손실')
plt.title('모델별 검증 손실 비교')
plt.xlabel('에포크 (Epochs)')
plt.ylabel('손실 (Loss)')
plt.legend()
plt.grid(True)

plt.show()